In [1]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

%config Completer.use_jedi = False

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
smoke_detection = tf.keras.models.load_model("smoking.h5")
font = cv2.FONT_HERSHEY_SIMPLEX
scale = 0.8

In [3]:
def predict(image):
    
    face_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_frame = cv2.resize(face_frame, (100, 100))
    face_frame = img_to_array(face_frame)
    face_frame = np.expand_dims(face_frame, axis=0)
    prediction = smoke_detection.predict(face_frame)
    
    return prediction[0][0]

In [6]:
def detector(gray_image, frame):
    
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5)
    
    for (x,y,w,h) in faces:

        roi_color = frame[y:y+h, x:x+w]
        smoke = predict(roi_color)

        
        if smoke > 0.5:
            
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(frame, text = 'Smoking', org =(x+50,y-100), fontFace =font, fontScale = scale, color =(0,255,0),
            thickness = 2)
        
        elif smoke < 0.5:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2)
            cv2.putText(frame, text = 'Not Smoking', org =(x+50,y-100), fontFace =font, fontScale = scale , color =(0,0,255),
            thickness = 2)
            
    return frame

In [9]:
video_cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = video_cap.read()
    ##### ret means return, frame menas from webcame we get the frame ######
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   ##### Convert RGB image to Gray image 
    detect = detector(gray_frame, frame)
    cv2.imshow("Video", detect)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
video_cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step
